# Create age grid animations for AGU 24 talk

Create animation of palaeo-age grids using GPlately/pyGMT for AGU talk.

In [1]:
import gplately
import pygmt
import numpy as np
from gplately import pygplates
import os

from plate_model_manager import PlateModelManager, PresentDayRasterManager

## Set plate model and get files

In [2]:
pm_manager = PlateModelManager()
model_name = "Muller2019"
data_dir = "plate-model-repo"

plate_model = pm_manager.get_model(model_name, data_dir=data_dir)

rotation_model = plate_model.get_rotation_model()
topology_filename = plate_model.get_topologies()
static_polygons = plate_model.get_static_polygons()

topology_features = pygplates.FeatureCollection()
for file in topology_filename:
    topology_features = pygplates.FeatureCollection(file)
    topology_features.add(topology_features)
    
coastlines = plate_model.get_layer('Coastlines')
continents = plate_model.get_layer('ContinentalPolygons')
COBs =  plate_model.get_layer('COBs')

model = gplately.PlateReconstruction(rotation_model, topology_features=topology_features, static_polygons=static_polygons)

downloading https://repo.gplates.org/webdav/pmm/muller2019/Rotations.zip
The local file(s) is/are still good. Will not download again at this moment.
downloading https://repo.gplates.org/webdav/pmm/muller2019/Topologies.zip
The local file(s) is/are still good. Will not download again at this moment.
downloading https://repo.gplates.org/webdav/pmm/muller2019/StaticPolygons.zip
The local file(s) is/are still good. Will not download again at this moment.
downloading https://repo.gplates.org/webdav/pmm/muller2019/Coastlines.zip
The local file(s) is/are still good. Will not download again at this moment.
downloading https://repo.gplates.org/webdav/pmm/muller2019/ContinentalPolygons.zip
The local file(s) is/are still good. Will not download again at this moment.
downloading https://repo.gplates.org/webdav/pmm/muller2019/COBs.zip
The local file(s) is/are still good. Will not download again at this moment.


### Download the agegrids

In [3]:
plate_model.get_rasters("AgeGrids", list(np.arange(0,251,1)));

downloading https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2019_Tectonics/Muller_etal_2019_Agegrids/Muller_etal_2019_Tectonics_v2.0_netCDF/Muller_etal_2019_Tectonics_v2.0_AgeGrid-0.ncdownloading https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2019_Tectonics/Muller_etal_2019_Agegrids/Muller_etal_2019_Tectonics_v2.0_netCDF/Muller_etal_2019_Tectonics_v2.0_AgeGrid-1.nc

downloading https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2019_Tectonics/Muller_etal_2019_Agegrids/Muller_etal_2019_Tectonics_v2.0_netCDF/Muller_etal_2019_Tectonics_v2.0_AgeGrid-2.nc
downloading https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2019_Tectonics/Muller_etal_2019_Agegrids/Muller_etal_2019_Tectonics_v2.0_netCDF/Muller_etal_2019_Tectonics_v2.0_AgeGrid-3.nc
downloading https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2019_Tectonics/Muller_etal_2019_Agegrids/Muller_etal_2019_Tectonics_v2.0_netCDF/Muller_etal_2019_Tectonics_v2.

In [5]:
agegrid_filename_prefix = '%s/%s/Rasters/AgeGrids/Muller_etal_2019_Tectonics_v2.0_AgeGrid-' % (data_dir, model_name)
agegrid_filename_ext = '.nc'

In [6]:
agegrid_filename_prefix

'plate-model-repo/Muller2019/Rasters/AgeGrids/Muller_etal_2019_Tectonics_v2.0_AgeGrid-'

### Set up plotting functions

In [20]:
def plot_age_through_time_robinson(time, plate_model, outdir, agegrid_filename_prefix, agegrid_filename_ext, central_meridian):

    age = time
    
    gplot = gplately.PlotTopologies(plate_model,
                                    coastlines=coastlines,
                                    continents=continents,
                                    COBs=COBs,
                                    time=age)

    age_grid = '%s%s%s' % (agegrid_filename_prefix, int(time), agegrid_filename_ext)

    # ----- parameters for plot
    region = 'd'
   
    width = 12
    projection = 'N%s/' % central_meridian
    x_offset = width + 1.2
    
    # plate boundary stuff
    plateboundary_width = '0.5p'
    plate_colour = 'black'
    subduction_zone_colour = 'dodgerblue3'
    ridge_colour = 'red'
    
    age_font = '14p,Helvetica,black'
    coastline_col = 'grey80'
    continent_col='grey70'
    
    label_font = '14p,Helvetica,black'
    label_offset = 'J0.1/0.0'
    label_position = 'TL'
    
    # ----- get plot things for gplately
    gdf_subduction_left, gdf_subduction_right = gplot.get_subduction_direction()
    
    # resolve plates - get topologies and subset to only closed plate boundaries
    gdf_topo = gplot.get_all_topologies(central_meridian=central_meridian)
    gdf_topo_plates = gdf_topo[(gdf_topo['feature_name'] == 'TopologicalClosedPlateBoundary')]
    gdf_topo_plates = gdf_topo_plates[~gdf_topo_plates.is_empty]

    # get ACTUAL transforms
    gdf_topo_sections = gplot.get_all_topological_sections()
    gdf_topo_transforms = gdf_topo_sections[(gdf_topo_sections['feature_type'] == pygplates.FeatureType.gpml_transform)]
    gdf_topo_transforms = gdf_topo_transforms.drop(columns='feature_type')  # get rid of the feature type column otherwise it will complain when plotting

    # -------------------------------------------------------------------------------
    # ------ plot
    fig = pygmt.Figure()
    pygmt.config(FONT_ANNOT=10, FONT_LABEL=8, FONT=10, MAP_TICK_PEN="0.75p", MAP_FRAME_PEN="0.75p", MAP_TICK_LENGTH_PRIMARY="4p")
    pygmt.makecpt(cmap="inferno", series=[0, 250, 5], reverse=True)
    fig.grdimage(grid=age_grid, region=region, projection="%s%sc" % (projection, width))
    
    fig.plot(data=gplot.get_coastlines(), fill=coastline_col, transparency=0) # COBs

    fig.plot(data=gdf_topo_plates.geometry, pen='%s,%s' % (plateboundary_width, plate_colour), frame="lrtb")
    fig.plot(data=gdf_subduction_left, pen='%s,%s' % (plateboundary_width, subduction_zone_colour), fill=subduction_zone_colour, style='f0.3/0.09+l+t')
    fig.plot(data=gdf_subduction_right, pen='%s,%s' % (plateboundary_width, subduction_zone_colour), fill=subduction_zone_colour, style='f0.3/0.09+r+t')
    fig.plot(data=gplot.get_ridges_and_transforms(), pen='%s,%s' % (plateboundary_width, ridge_colour))
    fig.plot(data=gdf_topo_transforms, pen='%s,palegreen3' % plateboundary_width)

    fig.basemap(region=region, projection="%s%sc" % (projection, width), frame="lrtb")
    fig.text(text='%s Ma' % age, position=label_position, no_clip=True, font=label_font, offset=label_offset)
    fig.colorbar(frame=["xa50f25+lAge (myr)"], position="JMR+o0.5c/0.2c+w5.25c/0.3c+n")
    
    fig.savefig('%s/age-%sMa.png' % (outdir, age), dpi=300)

# plot on robinson projection

includes age and scalebar

In [21]:
plate_model = model
central_meridian = 180

In [22]:
max_time = 250
times = np.arange(0, max_time + 1, 1)

In [24]:
plot_type = 'age_robinson'
outdir = 'figures_%s_%s' % (model_name, plot_type)
os.makedirs(outdir, exist_ok=True)

# plot files
for time in times:
    plot_age_through_time_robinson(time, plate_model, outdir, agegrid_filename_prefix, agegrid_filename_ext, central_meridian)


# copy 0 Ma
os.system('cp %s/age-0Ma.png %s/age0Ma.png' % (outdir, outdir))
os.system('ffmpeg -r 12 -start_number -%s -i %s/age%sMa.png  -pix_fmt yuv420p -vf "split=2[clr][bg];[bg]drawbox=c=white:t=fill[bg];[bg][clr]overlay,scale=trunc(iw/3.5)*2:trunc(ih/3.5)*2,pad=width=iw+50:height=ih+50:x=(ow-iw)/2:y=(oh-ih)/2:color=white" -y %s_age_robinson.mp4 < /dev/null'
 % (max_time, outdir, '%d', model_name))

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 15.0.7
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1692995812302/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1692995812302/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enab

0

-----

## plot on plate carree for title

In [31]:
def plot_age_through_time_pc(time, plate_model, outdir, agegrid_filename_prefix, agegrid_filename_ext, central_meridian):

    age = time
    
    gplot = gplately.PlotTopologies(plate_model,
                                    coastlines=coastlines,
                                    continents=continents,
                                    COBs=COBs,
                                    time=age)


    age_grid = '%s%s%s' % (agegrid_filename_prefix, int(time), agegrid_filename_ext)

    # ----- parameters for plot
    region = 'd'
   
    width = 12
    projection = 'Q%s/' % central_meridian
    x_offset = width + 1.2
    
    # plate boundary stuff
    plateboundary_width = '0.5p'
    plate_colour = 'black'
    subduction_zone_colour = 'dodgerblue3'
    ridge_colour = 'red'
    
    age_font = '14p,Helvetica,black'
    coastline_col = 'grey80'
    continent_col='grey70'
    
    label_font = '14p,Helvetica,black'
    label_offset = 'J0.15/0.25'
    label_position = 'TL'
    
    # ----- get plot things for gplately
    gdf_subduction_left, gdf_subduction_right = gplot.get_subduction_direction()
    
    # resolve plates - get topologies and subset to only closed plate boundaries
    gdf_topo = gplot.get_all_topologies(central_meridian=central_meridian)
    gdf_topo_plates = gdf_topo[(gdf_topo['feature_name'] == 'TopologicalClosedPlateBoundary')]
    gdf_topo_plates = gdf_topo_plates[~gdf_topo_plates.is_empty]

    gdf_topo_sections = gplot.get_all_topological_sections()
    gdf_topo_transforms = gdf_topo_sections[(gdf_topo_sections['feature_type'] == pygplates.FeatureType.gpml_transform)]
    gdf_topo_transforms = gdf_topo_transforms.drop(columns='feature_type')  # get rid of the feature type column otherwise it will complain when plotting

    # -------------------------------------------------------------------------------
    # ------ plot
    fig = pygmt.Figure()
    pygmt.config(FONT_ANNOT=10, FONT_LABEL=8, FONT=10, MAP_TICK_PEN="0.75p", MAP_FRAME_PEN="0.75p", MAP_TICK_LENGTH_PRIMARY="4p")
    pygmt.makecpt(cmap="inferno", series=[0, 220, 5], reverse=True)
    fig.grdimage(grid=age_grid, region=region, projection="%s%sc" % (projection, width))
    
    fig.plot(data=gplot.get_coastlines(), fill=coastline_col, transparency=0) # COBs

    fig.plot(data=gdf_topo_plates.geometry, pen='%s,%s' % (plateboundary_width, plate_colour), frame="lrtb")
    fig.plot(data=gdf_subduction_left, pen='%s,%s' % (plateboundary_width, subduction_zone_colour), fill=subduction_zone_colour, style='f0.3/0.09+l+t')
    fig.plot(data=gdf_subduction_right, pen='%s,%s' % (plateboundary_width, subduction_zone_colour), fill=subduction_zone_colour, style='f0.3/0.09+r+t')
    fig.plot(data=gplot.get_ridges_and_transforms(), pen='%s,%s' % (plateboundary_width, ridge_colour))
    
    fig.plot(data=gdf_topo_transforms, pen='%s,palegreen3' % plateboundary_width)
    
    fig.basemap(region=region, projection="%s%sc" % (projection, width), frame="lrtb")
    fig.savefig('%s/age-%sMa.png' % (outdir, age), dpi=300)

In [32]:
plot_type = 'age_pc'
outdir = 'figures_%s_%s' % (model_name, plot_type)
os.makedirs(outdir, exist_ok=True)

for time in times:
    plot_age_through_time_pc(time, plate_model, outdir, agegrid_filename_prefix, agegrid_filename_ext, central_meridian)

# copy 0 Ma
os.system('cp %s/age-0Ma.png %s/age0Ma.png' % (outdir, outdir))
os.system('ffmpeg -r 8 -start_number -%s -i %s/age%sMa.png  -pix_fmt yuv420p -vf "split=2[clr][bg];[bg]drawbox=c=white:t=fill[bg];[bg][clr]overlay,scale=trunc(iw/3.5)*2:trunc(ih/3.5)*2" -y %s_age_pc.mp4 < /dev/null' 
          % (max_time, outdir, '%d', model_name))

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 15.0.7
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1692995812302/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1692995812302/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enab

0